In [1]:
# tests for convert_Classic2FGN.py

In [2]:
from convert_Classic2FGN import convert_Classic2FGN

In [3]:
from __future__ import print_function

import numpy as np
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms

from Classic_MNIST_Net import Classic_MNIST_Net
from Feedforward_FGN_net import Feedforward_FGN_net
from test import test

In [4]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
use_cuda = True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

CUDA Available:  True


In [5]:
# load some data
# MNIST dataset and dataloader declaration
# transforms does both the conversion from 0-255 to 0-1
# and normalizes by the precomputed mean and std

batch_size = 32

mnist_train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../MNIST-dataset', train=True, download=False, 
                   transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))
            ])), 
        batch_size=batch_size, shuffle=True)

mnist_test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../MNIST-dataset', train=False, download=False, 
                   transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))
            ])), 
        batch_size=batch_size, shuffle=False)

In [6]:
# create models to be converted
classic_model = Classic_MNIST_Net(hidden_l_nums=[3])
classic_model.to(device)
fgn_model = Feedforward_FGN_net(28*28,10,[3])
fgn_model.to(device)

Feedforward_FGN_net(
  (hidden_layers): ModuleList(
    (0): FGN_layer()
  )
  (fl): FGN_layer()
)

In [7]:
# nll loss function
def classic_nll_loss_func(model, output, target):
    return F.nll_loss(output, target)

In [8]:
# nll loss function
def fgn_nll_loss_func(model, output, target):
#     # split output into pred and likelihoods
#     output, likelihood = output
    return F.nll_loss(output, target)    

In [9]:
# number of correct pred function for classic net
def classic_pred_func(output, target):
    output = output
    pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
    correct = pred.eq(target.long().view_as(pred)).sum().item()
    return correct

In [10]:
# number of correct pred function for fgnet
def fgn_pred_func(output, target):
#     # split output into pred and likelihoods
#     output,_ = output
    pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
    correct = pred.eq(target.long().view_as(pred)).sum().item()
    return correct

In [11]:
# before conversion
classic_test_res = test(classic_model, device, mnist_test_loader, loss_func=classic_nll_loss_func, verbose=True, pred_func=classic_pred_func)
fgn_test_res = test(fgn_model, device, mnist_test_loader, loss_func=fgn_nll_loss_func, verbose=True, pred_func=fgn_pred_func)

Test set - Average loss: 2.4534, Accuracy: 934/10000 (9%)
Test set - Average loss: 2.8420, Accuracy: 1566/10000 (16%)


In [12]:
# some fgnet dict values
print(fgn_model.state_dict())

OrderedDict([('hidden_layers.0.weights', tensor([[ -1.5012,   8.2789, -12.7157,  ..., -21.5142,  25.9261,  14.6363],
        [ 18.1362,   4.7484,   6.4255,  ...,   9.8120,  -9.6075, -20.5830],
        [-10.6995,   6.7656,  23.1998,  ...,  16.8342,   3.2759, -27.5183]],
       device='cuda:0')), ('hidden_layers.0.centers', tensor([[-0.0018,  0.0069, -0.0070,  ...,  0.0048, -0.0038, -0.0072],
        [-0.0043, -0.0086, -0.0025,  ...,  0.0092,  0.0004, -0.0005],
        [-0.0015, -0.0070, -0.0058,  ..., -0.0045,  0.0083, -0.0098]],
       device='cuda:0')), ('hidden_layers.0.sigs', tensor([787.4385, 785.4504, 787.0408], device='cuda:0')), ('fl.weights', tensor([[-0.5860,  0.9203,  1.4646],
        [ 1.2175,  0.4945, -1.7297],
        [-0.2195, -1.5886,  1.4347],
        [-0.5539, -1.7309,  0.6285],
        [-1.7040, -0.7200, -0.8957],
        [-1.4470,  0.8972, -0.5374],
        [ 0.6258,  0.8309,  0.1203],
        [-0.9475,  1.6287,  0.3470],
        [ 1.4616,  0.1671, -0.3667],
        

In [13]:
# CONVERT CALL
convert_Classic2FGN(classic_model=classic_model, fgn_model=fgn_model)

In [14]:
# after conversion
classic_test_res = test(classic_model, device, mnist_test_loader, loss_func=classic_nll_loss_func, verbose=True, pred_func=classic_pred_func)
fgn_test_res = test(fgn_model, device, mnist_test_loader, loss_func=fgn_nll_loss_func, verbose=True, pred_func=fgn_pred_func)

Test set - Average loss: 2.4534, Accuracy: 934/10000 (9%)
Test set - Average loss: 2.4037, Accuracy: 931/10000 (9%)


In [15]:
# somce fgnet dict values (should have changed)
print(fgn_model.state_dict())

OrderedDict([('hidden_layers.0.weights', tensor([[ 0.0257, -0.0090,  0.0183,  ..., -0.0214, -0.0161,  0.0165],
        [ 0.0206,  0.0172,  0.0077,  ...,  0.0005, -0.0324,  0.0327],
        [ 0.0147,  0.0283,  0.0036,  ...,  0.0063, -0.0146,  0.0045]],
       device='cuda:0')), ('hidden_layers.0.centers', tensor([[ 2.2244e-03, -7.8300e-04,  1.5831e-03,  ..., -1.8543e-03,
         -1.3937e-03,  1.4337e-03],
        [-1.4506e-03, -1.2149e-03, -5.4580e-04,  ..., -3.5653e-05,
          2.2848e-03, -2.3083e-03],
        [ 1.3063e-03,  2.5231e-03,  3.2489e-04,  ...,  5.6051e-04,
         -1.2986e-03,  3.9727e-04]], device='cuda:0')), ('hidden_layers.0.sigs', tensor([787.4385, 785.4504, 787.0408], device='cuda:0')), ('fl.weights', tensor([[ 0.4657,  0.4746,  0.2616],
        [-0.2940, -0.4533,  0.4877],
        [-0.5544, -0.5662, -0.0455],
        [-0.5734,  0.2629, -0.0180],
        [-0.5472, -0.3655,  0.0111],
        [-0.4185, -0.3685,  0.5030],
        [ 0.4346,  0.1642,  0.4301],
        

In [16]:
# Expected: results to be close to identical
# and without needing to reload the fgn_net